In [1]:
%pip install torch
%pip install PathLib
%pip install nltk
%pip install wakepy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
from torch import nn, Tensor, FloatTensor
import torch.nn.functional as F

In [3]:
import os
import pickle
import re
from collections import Counter

import nltk
import numpy as np
import torch
import pathlib

In [4]:
DATADIR = 'data'
MOD_VAR_DIR = 'models_and_variables'
TRAINED_MODELS_DIR = 'trained_models'
HP_TEXT_DIR = os.path.join(DATADIR, 'harry_potter_text')
dirs=[MOD_VAR_DIR, TRAINED_MODELS_DIR]
for dir in dirs:
    pathlib.Path(dir).mkdir(parents=True, exist_ok=True)

In [5]:
"""def generate_vocabulary(individual_words, include_special_tokens=False):
    condition_keys = sorted(individual_words)
    print(conditions.unique())
    result = dict(zip(condition_keys, range(len(condition_keys))))
    print(len(result))
    return result
"""

'def generate_vocabulary(individual_words, include_special_tokens=False):\n    condition_keys = sorted(individual_words)\n    print(conditions.unique())\n    result = dict(zip(condition_keys, range(len(condition_keys))))\n    print(len(result))\n    return result\n'

In [6]:
def generate_vocabulary(individual_words, min_threshold, include_special_tokens=False):
    """
    Return {token: index} for all train tokens (words) that occur min_threshold times or more,
        `index` should be from 0 to N, where N is a number of unique tokens in the resulting dictionary.
    """
    #create a list of words that happen min_threshold times or more in that string
    condition_keys = sorted([key for key, value in Counter(individual_words).items() if value >= min_threshold])
    #generate the vocabulary(dictionary)


    if not include_special_tokens:
        result = dict(zip(condition_keys, range(len(condition_keys))))
        return result
    else:
        result = dict(zip(condition_keys, range(3,len(condition_keys)+3)))
        orig = {"BOS": 0, "EOS": 1, "UNK": 2}
        orig.update(result)
        return orig

In [7]:
def get_hp_text(num_of_books=1):
    text_files = os.listdir(HP_TEXT_DIR)
    path_to_hp_text = os.path.join(MOD_VAR_DIR, "harry_potter_text.pkl")

    filepath = pathlib.PurePath(MOD_VAR_DIR, "harry_potter_text.pkl")

    counter=0
    if not os.path.exists(path_to_hp_text):
        all_text = ""
        for book in text_files:
             path_to_book = os.path.join(HP_TEXT_DIR, book)

             with open(path_to_book, "r", encoding="utf8") as f:
                text = f.readlines()

             text = [line for line in text if "Page" not in line]
             text = " ".join(text).replace("\n", "")
             text = [word for word in text.split(" ") if len(word) > 0]

             text = " ".join(text)
             text = re.sub("[^a-zA-Z0-9-_*.!,? \"\']", "", text)
             all_text+=text
             counter+=1
             if counter==num_of_books:
                 break

        with open(path_to_hp_text, 'wb') as handle:
            pickle.dump(all_text, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_hp_text, 'rb') as handle:
            all_text = pickle.load(handle)

    return all_text

In [8]:
def get_tokens():
    path_to_tokens = os.path.join(MOD_VAR_DIR, "harry_potter_tokens.pkl")
    if not os.path.exists(path_to_tokens):
        tokens=nltk.word_tokenize(get_hp_text())
        with open(path_to_tokens, 'wb') as handle:
            pickle.dump(tokens, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_tokens, 'rb') as handle:
            tokens = pickle.load(handle)
    return tokens

In [9]:
def get_2_vocabs():
    path_to_vocab= os.path.join(MOD_VAR_DIR, "harry_potter_vocab.pkl")
    path_to_inv_vocab = os.path.join(MOD_VAR_DIR, "harry_potter_inv_vocab.pkl")
    tokens= get_tokens()
    if not os.path.exists(path_to_vocab):
        vocab=generate_vocabulary(tokens, 1)
        inv_vocab = {v: k for k, v in vocab.items()}

        with open(path_to_vocab, 'wb') as handle:
            pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path_to_inv_vocab, 'wb') as handle:
            pickle.dump(inv_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_vocab, 'rb') as handle:
             vocab= pickle.load(handle)
        with open(path_to_inv_vocab, 'rb') as handle:
             inv_vocab= pickle.load(handle)

    return vocab, inv_vocab

In [10]:
class DataBuilder:
    def __init__(self, seq_len, tokens, word2idx={}, idx2word={}):
        self.seq_len = seq_len
        self.tokens = tokens
        self.number_of_tokens = len(tokens)
        self.char2idx=word2idx
        self.idx2char=idx2word

    def grab_random_sample(self):
        start = np.random.randint(0, self.number_of_tokens - self.seq_len)
        end = start + self.seq_len
        text_slice = self.tokens[start:end]

        input_text = text_slice[:-1]
        label = text_slice[1:]
        input_text = torch.tensor([self.char2idx[c] for c in input_text], dtype=torch.int32)
        label = torch.tensor([self.char2idx[c] for c in label], dtype=torch.int32)

        return input_text, label

    def grab_random_batch(self, batch_size):
        input_texts, labels = [], []

        for _ in range(batch_size):
            input_text, label = self.grab_random_sample()

            input_texts.append(input_text)
            labels.append(label)

        input_texts = torch.stack(input_texts)
        labels = torch.stack(labels)

        return input_texts, labels

In [11]:
nltk.download('punkt_tab')
import numpy as np
import torch
from torch import nn, Tensor, FloatTensor
import torch.nn.functional as F

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wilfr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
class LSTMForWordGeneration(nn.Module):
    def __init__(self, word2idx, idx2word,embedding_dim=128, hidden_size=256, n_layers=3):
        super().__init__()
        self.word2idx = word2idx
        self.idx2word = idx2word
        self.embedding_dim = embedding_dim
        self.num_words = len(word2idx)
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(self.num_words, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=n_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, self.num_words)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x)
        logits = self.fc(output)
        return logits

    def write(self, text, max_words, greedy=False):

        idx = torch.tensor([self.word2idx[w] for w in text])
        hidden = torch.zeros(self.n_layers, self.hidden_size)
        cell = torch.zeros(self.n_layers, self.hidden_size)

        for i in range(max_words):

            if i == 0:
                selected_idx = idx
            else:
                selected_idx = idx[-1].unsqueeze(0)

            x = self.embedding(selected_idx)
            out, (hidden, cell) = self.lstm(x, (hidden, cell))

            out = self.fc(out)

            if len(out) > 1:
                out = out[-1, :].unsqueeze(0)

            probs = self.softmax(out)

            if greedy:
                idx_next = torch.argmax(probs).squeeze(0)
            else:
                idx_next = torch.multinomial(probs, num_samples=1).squeeze(0)

            idx = torch.cat([idx, idx_next])

        gen_string = [self.idx2word[int(w)] for w in idx]
        gen_string = " ".join(gen_string)

        return gen_string

In [13]:
from pathlib import Path
from torch import optim, nn
import pandas as pd

def train(model, word2idx, idx2word, tokens, config):
    training_data = {
        "model":[],
        "iteration":[],
        "training data length":[],
        "loss": [],
        "generated text": []
    }
    name=model.__class__.__name__
    new_dir =Path(os.path.join(TRAINED_MODELS_DIR, name))
    new_dir.mkdir(parents=True, exist_ok=True)
    iterations = config["iterations"]
    max_len = config["max_len"]
    evaluate_interval = config["evaluate_interval"]
    embedding_dim = config["embedding_dim"]
    hidden_size = config["hidden_size"]
    n_layers = config["n_layers"]
    lr = config["lr"]
    batch_size = config["batch_size"]
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    dataset = DataBuilder(max_len, tokens, word2idx, idx2word)
    model.train()
    for iteration in range(iterations):
        input_texts, labels = dataset.grab_random_batch(batch_size=batch_size)
        input_texts, labels = input_texts, labels
        optimizer.zero_grad()
        output = model(input_texts)
        output = output.transpose(1,2)

        loss = loss_fn(output, labels.long())

        loss.backward()
        optimizer.step()

        if iteration % evaluate_interval == 0:
            model.eval()
            torch.no_grad()
            print("--------------------------------------")
            print(f"training text length {max_len}")
            print(f"Iteration {iteration}")
            print(f"Loss {loss.item()}")
            generated_text = model.write(["Spells"], max_words=50)
            print(generated_text)
            print("--------------------------------------")
            training_data["model"].append(name)
            training_data["iteration"].append(iteration)
            training_data["training data length"].append(max_len)
            training_data["loss"].append(loss.item())
            training_data["generated text"].append(generated_text)
            torch.enable_grad()
            model.train()

    training_data=pd.DataFrame(training_data)
    training_data.set_index(["model", "iteration"])
    training_data.to_csv(os.path.join(TRAINED_MODELS_DIR,name,"training_data.csv"), encoding="utf8")
    config_data=pd.DataFrame(list(config.items()), columns=["key", "value"])
    config_data.to_csv(os.path.join(TRAINED_MODELS_DIR,name,"config_data.csv"), encoding="utf8", index=False)
    torch.save(model.state_dict(), os.path.join(TRAINED_MODELS_DIR,name+"/model_state.pth"))


In [14]:
def get_config():
    config={}
    config["iterations"]=300
    config["max_len"]=20
    config["evaluate_interval"]=30
    config["embedding_dim"]=128
    config["hidden_size"]=256
    config["n_layers"]=4
    config["lr"]=0.003
    config["batch_size"]=64
    config["bidirectional"]=False
    return config

In [15]:
text = get_hp_text()
tokens = get_tokens()
word2idx, idx2word = get_2_vocabs()
config=get_config()

In [16]:
from wakepy import keep 
model = LSTMForWordGeneration(word2idx, idx2word, embedding_dim=config["embedding_dim"],
                                      hidden_size=config["hidden_size"], n_layers=config["n_layers"])
with keep.running():
    train(model, word2idx, idx2word, tokens, config)


--------------------------------------
training text length 20
Iteration 0
Loss 8.83820629119873
Spells right-handed with Percy promised Platform Having secret easier trash hanger swiftly ankles Scared Jordans coins wrong cabbages passed standard stopped jinxing collar everything badge forgotten moleskin Wheres VANASHIG constrictors nearby care Baruffio stick folded steeling mustve armed Remembrall Tut swollen stick lion-fish GET heels Fortunately crowds horribly shouted hut escape
--------------------------------------
--------------------------------------
training text length 20
Iteration 30
Loss 6.69385290145874
Spells hes Hogwarts of theyre in sort barrier the join , is during Ive high even into course want clicked at of empty Can . the woke of I him about But starting two never nothing kept Going us , the of , , said one crumpled there you , hear
--------------------------------------
--------------------------------------
training text length 20
Iteration 60
Loss 6.5342712402343

In [ ]:
import t
import torch


In [ ]:
max = 7
min = 3
data_tensor = (max-min)*torch.rand((5, 2)) + min

tensor([[3.4339, 6.5122],
        [6.1769, 6.7556],
        [3.9282, 5.6291],
        [4.4330, 3.7920],
        [5.7043, 3.9537]])


In [36]:
q_index = 10
# Convert the index into a one-hot-coded vector with the same length as the number of samples
# This will be our "query" vector (q)
q_one_hot_vec = F.one_hot(torch.tensor([2]), 5)

# Create a unique one-hot-coded vector for every image in our dataset
# These will be our "key" vectors (k)
k_one_hot = F.one_hot(torch.arange(5), 5)

# Randomly shuffle the keys and dataset to demonstrate that we can find the target image
# even in a randomly organized dataset
rand_perm = torch.randperm(5)
print(f"rand_perm {rand_perm}")
print(f"data_tensor {data_tensor}")
data_tensor = data_tensor[rand_perm]
print(f"perm data_tensor {data_tensor}")
print(f"k_one_hot {k_one_hot}")
k_one_hot = k_one_hot[rand_perm]
print(f"perm k_one_hot {k_one_hot}")

print(f"q_one_hot_vec {q_one_hot_vec} ")

print(f"data_tensor {data_tensor}")

rand_perm tensor([2, 3, 4, 1, 0])
data_tensor tensor([[3.4339, 6.5122],
        [6.1769, 6.7556],
        [3.9282, 5.6291],
        [4.4330, 3.7920],
        [5.7043, 3.9537]])
perm data_tensor tensor([[3.9282, 5.6291],
        [4.4330, 3.7920],
        [5.7043, 3.9537],
        [6.1769, 6.7556],
        [3.4339, 6.5122]])
k_one_hot tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])
perm k_one_hot tensor([[0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0]])
q_one_hot_vec tensor([[0, 0, 1, 0, 0]]) 
data_tensor tensor([[3.9282, 5.6291],
        [4.4330, 3.7920],
        [5.7043, 3.9537],
        [6.1769, 6.7556],
        [3.4339, 6.5122]])


In [42]:
q = F.one_hot(torch.tensor([2]), 5)
k = F.one_hot(torch.arange(5), 5)
data = torch.randint(2, 8, (5,2 ))
print("q")
print(q)
print("k")
print(k)
print("data")
print(data)

q_times_k=torch.mm(q,k.T)
print("q_times_k")
print(q_times_k)
print("q_times_k_times_data")
print(torch.mm(q_times_k, data))

q
tensor([[0, 0, 1, 0, 0]])
k
tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])
data
tensor([[2, 3],
        [4, 6],
        [4, 2],
        [5, 2],
        [6, 5]])
q_times_k
tensor([[0, 0, 1, 0, 0]])
q_times_k_times_data
tensor([[4, 2]])


In [48]:
q= torch.FloatTensor([0,0.3,0.2,0.4,0.1]).unsqueeze(0)
k = F.one_hot(torch.arange(5), 5).to(torch.float)
data = data.to(torch.float)

print("q")
print(q)
print("k")
print(k)
print("data")
print(data)

q_times_k=torch.mm(q,k.T)
print("q_times_k")
print(q_times_k)
print("q_times_k_times_data")
print(torch.mm(q_times_k, data))

q
tensor([[0.0000, 0.3000, 0.2000, 0.4000, 0.1000]])
k
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
data
tensor([[2., 3.],
        [4., 6.],
        [4., 2.],
        [5., 2.],
        [6., 5.]])
q_times_k
tensor([[0.0000, 0.3000, 0.2000, 0.4000, 0.1000]])
q_times_k_times_data
tensor([[4.6000, 3.5000]])


In [ ]:
class MyAttention(nn.Module):
    def __init__(self, d_in,d_out):
        super().__init__()
        self.d_in=d_in
        self.d_out=d_out
        self.Q=nn.Linear(4*d_in,d_out)
        self.K=nn.Linear(d_in,d_out)
        self.V=nn.Linear(d_in,d_out)
    def forward(self,q ,x):
        queries=self.Q(q)
        keys = self.K(x)
        values = self.V(x)
        scores = torch.bmm(queries, keys.transpose(1,2))
        scores = scores/ (self.d_out**0.5)
        attention = F.softmax(scores,dim=2)
        hidden_states= torch.bmm(attention,values)
        return hidden_states